In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Генерація даних
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Побудова моделі MLP
model = Sequential([
    Dense(64, activation='relu', input_shape=(10,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Компіляція
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Навчання
model.fit(X_train, y_train, epochs=20, batch_size=10, validation_data=(X_test, y_test))

# Оцінка
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Точність (Adam): {accuracy:.2f}")


ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Генерація даних
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Побудова моделі MLP
model = Sequential([
    Dense(64, activation='relu', input_shape=(10,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Компіляція
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Навчання
model.fit(X_train, y_train, epochs=20, batch_size=10, validation_data=(X_test, y_test))

# Оцінка
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Точність (Adam): {accuracy:.2f}")


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import tensorflow as tf
print(tf.__version__)


ModuleNotFoundError: No module named 'tensorflow'

In [1]:
%pip uninstall tensorflow
%pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl (375.9 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\TZ\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [2]:
%pip cache purge
%pip install tensorflow


Files removed: 470 (545.9 MB)
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/375.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/375.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/375.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/375.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/375.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/375.9 MB 492.8 kB/s eta 0:12:42
   ---------------------------------------- 0.5/375.9 MB 492.8 kB/s eta 0:12:42
   ---------------------------------------- 0.5/375.9 MB 492.8 kB/s eta 0:12:42
   ---------------------------------------- 0.5/375.9 MB 492.8 kB/s eta 0:12:42
   ---------------------------------------- 0.8/375.9 MB 372.6 kB/s eta 0:16:47
   ---------------------------------------- 0.8/375.9 MB 372.6 kB/s eta 0:16:47
   ---------------------------------------- 0.8/375.9 MB 372.6 kB/s e

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\TZ\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [1]:
%pip uninstall tensorflow


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install tensorflow


  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl (375.9 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\TZ\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [4]:

%pip install tensorflow


  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl (375.9 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\TZ\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Створення нейромережі
model = Sequential([
    Dense(64, activation='relu', input_shape=(10,)),  # Вхідний шар
    Dense(32, activation='relu'),                     # Прихований шар
    Dense(1, activation='sigmoid')                    # Вихідний шар (бінарна класифікація)
])

# Компільовуємо модель
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Виведення структури моделі
model.summary()

ModuleNotFoundError: No module named 'tensorflow.python'